#### **Data/Analysis**

  The dataset for building the machine learning model is the accident data from Settle City. The dataset includes 37 attributes and 194,673 rows. The dataset assigns each accident with a unique ID number and provides information such as locations, time, and type of collision. Most importantly, the dataset also includes critical information on the factors that are related to the accident. For example, the weather condition column, “WEATHER”, gives a general idea of the weather during the time of the accident—cloudy, clear, or other conditions. “SPEEDING” column identifies weather the driver exceeds the speed limit. “UNDERINFL” explains whether the driver drinks and drives. 

In [ ]:
import pandas as pd
import numpy as np 
!wget -O collision.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv

In [3]:
df = pd.read_csv('collision.csv')
df.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


“SEVERITY” is identified as the dependent variable of this dataset or the targeted variables, as we wish to predict this variable with the machine learning model. Therefore, a new dataset is created with only attributes necessary for the data analysis. The new dataset includes both dependent and independent variables. For this research, 8 independents variables are identified.  There are ADDRTYPE(the general description of collision location), INATTENTIONIND(whether the driver pays attention), UNDERINFL(whether the driver is under influence), WEATHER(the weather condition), ROADCOND(the road condition), PEDROWNOTGRNT(the right of pedestrian is granted or not), SPEEDING(the speed limit). 

In [4]:
new_df=df[['ADDRTYPE','INATTENTIONIND','UNDERINFL','WEATHER','ROADCOND','LIGHTCOND','PEDROWNOTGRNT','SPEEDING','SEVERITYDESC']]
new_df.head()

,ADDRTYPE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,SEVERITYDESC
0,Intersection,NaN,N,Overcast,Wet,Daylight,NaN,NaN,Injury Collision
1,Block,NaN,0,Raining,Wet,Dark - Street Lights On,NaN,NaN,Property Damage Only Collision
2,Block,NaN,0,Overcast,Dry,Daylight,NaN,NaN,Property Damage Only Collision
3,Block,NaN,N,Clear,Dry,Daylight,NaN,NaN,Property Damage Only Collision
4,Intersection,NaN,0,Raining,Wet,Daylight,NaN,NaN,Injury Collision


In INATTENTIONIND, UNDERINFL, PEDROWNOTGRNT, SPEEDING, only Y is recorded. So I fill all the null values with “N” for data cleaning. Also, under the ROADCOND, LIGHTCOND, WEATHER  columns, a missing value is recorded as “Other”. 

In [7]:
new_df['INATTENTIONIND'].fillna('N',inplace=True)
new_df['SPEEDING'].fillna('N',inplace=True)
new_df['ADDRTYPE'].fillna('Other',inplace=True)
new_df['PEDROWNOTGRNT'].fillna('N',inplace=True)
new_df['UNDERINFL'] = new_df['UNDERINFL'].replace(['0'],'N')
new_df['UNDERINFL'] = new_df['UNDERINFL'].replace(['1'],'Y')
new_df['UNDERINFL'].fillna('N',inplace=True)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
new_df['WEATHER'].fillna('Other',inplace=True)
new_df['ROADCOND'].fillna('Other',inplace=True)
new_df['LIGHTCOND'].fillna('Other',inplace=True)

I also group the light condition into dark, normal, and other. For the weather condition, there are 4 groups--clear, cloudy, raining,Fog/Smog/Smoke and others. Blowing Sand/Dirt and Severe Crosswind rows are dropped because they only contain less than 100 rows. 

In [9]:
new_df['LIGHTCOND'] = new_df['LIGHTCOND'].replace(['Dark - Street Lights On'],'Dark')
new_df['LIGHTCOND'] = new_df['LIGHTCOND'].replace(['Dark - No Street Lights'],'Dark')
new_df['LIGHTCOND'] = new_df['LIGHTCOND'].replace(['Dark - Street Lights Off'],'Dark')
new_df['LIGHTCOND'] = new_df['LIGHTCOND'].replace(['Dark - Unknown Lighting'],'Dark')
new_df['LIGHTCOND'] = new_df['LIGHTCOND'].replace(['Dusk'],'Dark')
new_df['LIGHTCOND'] = new_df['LIGHTCOND'].replace(['Daylight'],'Normal')
new_df['LIGHTCOND'] = new_df['LIGHTCOND'].replace(['Dawn'],'Normal')
new_df['LIGHTCOND'] = new_df['LIGHTCOND'].replace(['Unknown'],'Other')
new_df['LIGHTCOND'].value_counts()

Normal    118598
Dark       57125
Other      18869
Name: LIGHTCOND, dtype: int64

In [ ]:
new_df['WEATHER']=new_df['WEATHER'].replace(['Unknown'],'Other')
new_df['WEATHER']=new_df['WEATHER'].replace(['Overcast'],'Cloudy')
new_df['WEATHER']=new_df['WEATHER'].replace(['Partly Cloudy'],'Cloudy')
new_df['WEATHER']=new_df['WEATHER'].replace(['Sleet/Hail/Freezing Rain'],'Raining')
new_df=new_df[new_df['WEATHER']!='Blowing Sand/Dirt']
new_df=new_df[new_df['WEATHER']!='Severe Crosswind']

In [10]:
new_df['WEATHER'].value_counts()

Clear             111135
Raining            33258
Cloudy             27719
Other              21004
Snowing              907
Fog/Smog/Smoke       569
Name: WEATHER, dtype: int64

For the road condition, there are 4 groups--dry, wet, Ice/Snow,and others. Sand/Mud/Dirt and Oil rows are dropped because they only contain less than 100 rows. 

In [11]:
new_df['ROADCOND']=new_df['ROADCOND'].replace(['Unknown'],'Other')
new_df['ROADCOND']=new_df['ROADCOND'].replace(['Standing Water'],'Wet')
new_df['ROADCOND']=new_df['ROADCOND'].replace(['Ice'],'Ice/Snow')
new_df['ROADCOND']=new_df['ROADCOND'].replace(['Snow/Slush'],'Ice/Snow')
new_df=new_df[new_df['ROADCOND']!='Sand/Mud/Dirt']
new_df=new_df[new_df['ROADCOND']!='Oil']

In [12]:
new_df['ROADCOND'].value_counts()

Dry         124468
Wet          47569
Other        20218
Ice/Snow      2198
Name: ROADCOND, dtype: int64

This concludes the data section of the report. 